In [1]:
import json
import csv

In [2]:
input_file = 'data/dengue_data.txt'
output_file = 'data/google_dengue.json'

In [3]:
country_names = []
with open("data/world-110m-country-names.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    for row in rd:
#         print(row[1])
        country_names.append(row[1])
country_names = set(country_names)

In [4]:
country_names

{'Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antarctica',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia, Plurinational State of',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Congo, the Democratic Republic of the',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Falkland Islands (Malvinas)',
 'Fiji',
 'Finland',
 'France',
 'French Southern Territories',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Greenland',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Ho

In [5]:
data_names = []
with open(input_file) as fd:
    rd = csv.reader(fd, delimiter=",")
    for row in rd:
        if row[0] == 'Date':
            data_names = row
            data_names.remove('Date')
            data_names = set(data_names)
            break

In [6]:
data_names

{'Argentina',
 'Bolivia',
 'Brazil',
 'India',
 'Indonesia',
 'Mexico',
 'Philippines',
 'Singapore',
 'Thailand',
 'Venezuela'}

In [7]:
def string_convert(input_string):
    if input_string == 'Bolivia':
        return 'Bolivia, Plurinational State of'
    if input_string == 'Venezuela':
        return 'Venezuela, Bolivarian Republic of'
    else:
        return input_string

In [8]:
for data_name in data_names:
    if string_convert(data_name) not in country_names:
        print(data_name)

Singapore


In [9]:
country_ids = {}
with open("data/world-110m-country-names.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    for row in rd:
        if row[1] == 'name':
            continue
        country_id = int(row[0])
        country_name = row[1]
        country_ids[country_name] = country_id

In [10]:
dictionary = {}
country_mapping = {}
with open(input_file) as fd:
    rd = csv.reader(fd, delimiter=",")
    week_num = 1
    prev_year = 2002
    for row in rd:
        if row[0] == 'Date':
            dictionary['countries'] = {}
            for i in range(1,len(row)):
                if string_convert(row[i]) not in dictionary['countries']:
                    country_mapping[i] = string_convert(row[i])
                    dictionary['countries'][string_convert(row[i])] = {}
                    dictionary['countries'][string_convert(row[i])]['name'] = string_convert(row[i])
                    dictionary['countries'][string_convert(row[i])]['time'] = {}
                    if string_convert(row[i]) in country_ids:
                        dictionary['countries'][string_convert(row[i])]['id'] = country_ids[string_convert(row[i])]
                    else:
                        dictionary['countries'][string_convert(row[i])]['id'] = -9999
        else:
            if int(row[0].split('-')[0]) != prev_year:
                week_num = 1
                prev_year = int(row[0].split('-')[0])
            
            attribute = 'weekly dengue activity estimates for year ' + str(prev_year)
            
            for i in range(1,len(row)):
                if week_num not in dictionary['countries'][country_mapping[i]]['time']:
                    dictionary['countries'][country_mapping[i]]['time'][week_num] = {}
                if prev_year not in dictionary['countries'][country_mapping[i]]['time'][week_num]:
                    if row[i] == '':
                        dictionary['countries'][country_mapping[i]]['time'][week_num][attribute] = 0.0
                    else:
                        dictionary['countries'][country_mapping[i]]['time'][week_num][attribute] = float(row[i])
                    
            week_num += 1

In [11]:
dictionary

{'countries': {'Argentina': {'id': 32,
   'name': 'Argentina',
   'time': {1: {'weekly dengue activity estimates for year 2003': 0.0,
     'weekly dengue activity estimates for year 2004': 0.019,
     'weekly dengue activity estimates for year 2005': 0.012,
     'weekly dengue activity estimates for year 2006': 0.01,
     'weekly dengue activity estimates for year 2007': 0.01,
     'weekly dengue activity estimates for year 2008': 0.012,
     'weekly dengue activity estimates for year 2009': 0.014,
     'weekly dengue activity estimates for year 2010': 0.482,
     'weekly dengue activity estimates for year 2011': 0.041,
     'weekly dengue activity estimates for year 2012': 0.034,
     'weekly dengue activity estimates for year 2013': 0.053,
     'weekly dengue activity estimates for year 2014': 0.043,
     'weekly dengue activity estimates for year 2015': 0.034},
    2: {'weekly dengue activity estimates for year 2003': 0.0,
     'weekly dengue activity estimates for year 2004': 0.017

In [12]:
with open(output_file, 'w') as fp:
    json.dump(dictionary, fp)